In [1]:
# 필수 데이터 분석 및 시각화, 백테스팅 라이브러리 설치
%pip install pandas numpy plotly bt quantstats requests python-dotenv nbformat openai

Note: you may need to restart the kernel to use updated packages.


## Step 0. KIS API로 ETF 일별 종가 데이터 준비하기

이 노트북에서는 **KIS 일봉 차트 API**를 통해 ETF의 과거 일별 종가 데이터를 직접 가져옵니다.

- 최초 실행 시:
  - `data/{종목코드}/NAV.csv` 파일이 없으면  
    👉 KIS API에서 과거 일봉 데이터를 받아와 `NAV.csv`로 저장합니다.
- 이후 실행 시:
  - 같은 파일이 있으면 API를 다시 호출하지 않고,  
    👉 캐시된 `NAV.csv`를 그대로 사용합니다.

> ⚠️ 가격 데이터는 3·5강에서 사용하던 **KIS 차트 API 옵션(`FID_ORG_ADJ_PRC`)** 과 동일하게 설정되어 있어,  
> 분배금/수정주가 처리 방식도 강의에서 설명한 것과 같은 기준으로 해석하면 됩니다.


## 1. KIS 환경 설정 & 공통 유틸

### 1-1. 환경변수 & 공통 헤더

In [2]:
from pathlib import Path
from datetime import datetime, timedelta

import json
import numpy as np
import pandas as pd
import plotly.express as px
import bt
import quantstats as qs
import requests
from openai import OpenAI

from kis.env import (
    ensure_kis_token,
    KIS_URL_BASE,
    get_api_headers,
    to_float,
    DATA_DIR,
)
from kis.meta import get_etf_name

client = OpenAI()

### 2-1. ETF 이름 가져오기 (MST → KIS 폴백)

In [3]:
DEFAULT_ETF_CODES = ["069500", "153130", "329750"]
for code in DEFAULT_ETF_CODES:
    print(code, "→", get_etf_name(code))


✅ ETF 이름(MST): 069500 → KODEX 200
069500 → KODEX 200
✅ ETF 이름(MST): 153130 → KODEX 단기채권
153130 → KODEX 단기채권
✅ ETF 이름(MST): 329750 → TIGER 미국달러단기채권액티브
329750 → TIGER 미국달러단기채권액티브


### 2-2. 현재가 · NAV · AUM · 추적오차

In [4]:
def fetch_etf_price_snapshot(etf_code: str) -> dict:
    """
    이 챕터(시세 스냅샷 실습) 전용 함수.
    kis 패키지의 공통 유틸만 가져다 쓰고,
    실제 비즈니스 로직은 여기서 구현.
    """
    # 방어적으로 토큰 한 번 체크
    ensure_kis_token()

    url = f"{KIS_URL_BASE}/uapi/etfetn/v1/quotations/inquire-price"
    params = {"FID_COND_MRKT_DIV_CODE": "J", "FID_INPUT_ISCD": etf_code}
    headers = get_api_headers("FHPST02400000")

    res = requests.get(url, headers=headers, params=params, timeout=5)
    data = res.json()

    if data.get("rt_cd") != "0":
        raise RuntimeError(f"[{etf_code}] 시세 조회 실패: {data.get('msg1')}")

    o = data["output"]

    current_price       = to_float(o.get("stck_prpr"))
    volume              = to_float(o.get("acml_vol"))
    prev_volume         = to_float(o.get("prdy_vol"))
    nav                 = to_float(o.get("nav"))
    nav_change          = to_float(o.get("nav_prdy_vrss"))
    nav_change_pct      = to_float(o.get("nav_prdy_ctrt"))
    listing_date_raw    = o.get("stck_lstn_date")
    aum_from_api        = to_float(o.get("etf_ntas_ttam"))
    aum_circulating_api = to_float(o.get("etf_crcl_ntas_ttam"))
    circulating_shares  = to_float(o.get("etf_crcl_stcn"))
    tracking_error_api  = to_float(o.get("trc_errt"))

    # 상장일/경과 기간
    listing_date = None
    years_since = None
    days_since = None
    if listing_date_raw:
        listing_date = datetime.strptime(listing_date_raw, "%Y%m%d")
        days_since = (datetime.now() - listing_date).days
        years_since = days_since / 365.25

    # AUM 계산
    aum_from_calc = None
    if nav and circulating_shares:
        aum_from_calc = nav * circulating_shares

    if aum_from_calc:
        aum_total = aum_from_calc
    elif aum_from_api:
        aum_total = aum_from_api * 100_000_000  # 억 단위 보정
    else:
        aum_total = None

    aum_billion = aum_total / 100_000_000 if aum_total else None
    aum_trillion = aum_billion / 10_000 if aum_billion else None

    # 괴리율
    premium = None
    if current_price and nav and nav > 0:
        premium = (current_price - nav) / nav * 100

    # 오늘 거래대금
    turnover_billion = None
    if current_price and volume:
        turnover_billion = current_price * volume / 100_000_000

    snapshot = {
        "etf_code": etf_code,
        "etf_name": get_etf_name(etf_code),
        "listing_date": listing_date.strftime("%Y-%m-%d") if listing_date else None,
        "days_since_listing": days_since,
        "years_since_listing": round(years_since, 2) if years_since else None,
        "current_price": current_price,
        "volume": volume,
        "prev_volume": prev_volume,
        "nav": nav,
        "nav_change": nav_change,
        "nav_change_pct": nav_change_pct,
        "premium_pct": premium,
        "turnover_billion": turnover_billion,
        "aum_total": aum_total,
        "aum_billion": aum_billion,
        "aum_trillion": aum_trillion,
        "tracking_error_api": tracking_error_api,
    }
    return snapshot

### 2-3. 구성 종목 리스트 + 비중

In [5]:
def fetch_etf_components(etf_code: str) -> pd.DataFrame:
    """
    구성 종목 코드/이름/비중/현재가/평가액/시총(있으면) 가져오기
    """
    ensure_kis_token()
    url = f"{KIS_URL_BASE}/uapi/etfetn/v1/quotations/inquire-component-stock-price"
    params = {
        "FID_COND_MRKT_DIV_CODE": "J",
        "FID_INPUT_ISCD": etf_code,
        "FID_COND_SCR_DIV_CODE": "11216"
    }
    headers = get_api_headers("FHKST121600C0")

    res = requests.get(url, headers=headers, params=params, timeout=5)
    data = res.json()

    if data.get("rt_cd") != "0":
        raise RuntimeError(f"[{etf_code}] 구성 종목 조회 실패: {data.get('msg1')}")

    rows = []
    for row in data.get("output2", []):
        code      = row.get("stck_shrn_iscd")
        name      = row.get("hts_kor_isnm")
        weight    = to_float(row.get("etf_cnfg_issu_rlim"))
        price     = to_float(row.get("stck_prpr"))
        valuation = to_float(row.get("etf_vltn_amt"))
        mcap_etf  = to_float(row.get("hts_avls"))

        rows.append({
            "code": code,
            "name": name,
            "weight_pct": weight,
            "price": price,
            "valuation": valuation,
            "market_cap": mcap_etf,
            "is_domestic": code.isdigit() and len(code) == 6 if code else False
        })

    df = pd.DataFrame(rows).dropna(subset=["weight_pct"]).sort_values("weight_pct", ascending=False)
    return df.reset_index(drop=True)


comp_069500 = fetch_etf_components("069500")
comp_069500.head()

✅ 기존 토큰 유효함


,code,name,weight_pct,price,valuation,market_cap,is_domestic
0,005930,삼성전자,26.00,102800.0,704831400.0,6085388.0,True
1,000660,SK하이닉스,15.89,524000.0,441669000.0,3814732.0,True
2,105560,KB금융,2.18,124700.0,59754700.0,475683.0,True
3,005380,현대차,1.91,261500.0,52787500.0,535442.0,True
4,035420,NAVER,1.85,263500.0,49841000.0,413307.0,True


## 3. KIS 일봉 차트 API로 NAV 시계열 캐싱

In [6]:
BASE_DATA_DIR = DATA_DIR


def download_kis_daily_prices(etf_code: str, start: str = "2010-01-01") -> pd.DataFrame:
    """
    - start 날짜 이후 ~ 오늘까지 일별 종가를 받아와서
    - index = Date, column = Close 형태의 DataFrame 반환
    """
    ensure_kis_token()

    start_dt = datetime.fromisoformat(start)
    end_dt = datetime.now()
    all_records = []

    cur_end = end_dt
    while cur_end >= start_dt:
        cur_start = max(start_dt, cur_end - timedelta(days=365))

        url = f"{KIS_URL_BASE}/uapi/domestic-stock/v1/quotations/inquire-daily-itemchartprice"
        params = {
            "FID_COND_MRKT_DIV_CODE": "J",
            "FID_INPUT_ISCD": etf_code,
            "FID_INPUT_DATE_1": cur_start.strftime("%Y%m%d"),
            "FID_INPUT_DATE_2": cur_end.strftime("%Y%m%d"),
            "FID_PERIOD_DIV_CODE": "D",
            "FID_ORG_ADJ_PRC": "0",   # 강의에서 쓰던 옵션 유지
        }
        headers = get_api_headers("FHKST03010100")

        res = requests.get(url, headers=headers, params=params, timeout=10)
        data = res.json()

        if data.get("rt_cd") != "0":
            raise RuntimeError(f"[{etf_code}] 일봉 조회 실패: {data.get('msg1')}")

        for row in data.get("output2", []):
            d = row.get("stck_bsop_date")
            close = to_float(row.get("stck_clpr"))
            if d and close is not None:
                all_records.append(
                    (pd.to_datetime(d, format="%Y%m%d"), close)
                )

        cur_end = cur_start - timedelta(days=1)

    if not all_records:
        raise ValueError(f"[{etf_code}] 일봉 데이터 없음")

    df = (pd.DataFrame(all_records, columns=["Date", "Close"])
            .drop_duplicates("Date")
            .sort_values("Date")
            .set_index("Date"))

    print(f"[INFO] {etf_code} 기간: {df.index.min().date()} ~ {df.index.max().date()} (영업일 {len(df)}개)")
    return df


def load_or_download_nav(etf_code: str, start: str = "2010-01-01") -> pd.DataFrame:
    etf_dir = BASE_DATA_DIR / etf_code
    etf_dir.mkdir(parents=True, exist_ok=True)
    csv_path = etf_dir / "NAV.csv"

    if csv_path.exists():
        df = pd.read_csv(csv_path, index_col=0, parse_dates=True)
        print(f"[LOAD] {etf_code} NAV.csv: {df.index.min().date()} ~ {df.index.max().date()} ({len(df)}일)")
        return df

    print(f"[DOWNLOAD] {etf_code} NAV 데이터 다운로드 시작...")
    df = download_kis_daily_prices(etf_code, start=start)
    df.to_csv(csv_path)
    print(f"[SAVE] {csv_path} 저장 완료")
    return df


df_069500 = load_or_download_nav("069500", start="2010-01-01")
df_153130 = load_or_download_nav("153130", start="2012-01-01")
df_329750 = load_or_download_nav("329750", start="2010-01-01")

df_069500.head()

[LOAD] 069500 NAV.csv: 2010-06-24 ~ 2025-11-26 (1600일)
[LOAD] 153130 NAV.csv: 2012-06-27 ~ 2025-11-26 (1400일)
[LOAD] 329750 NAV.csv: 2019-07-24 ~ 2025-11-26 (683일)


,Close
Date,
2010-06-24,17421.0
2010-06-25,17257.0
2010-06-28,17326.0
2010-06-29,17116.0
2010-06-30,16963.0


## 4. 시각화 - 시작점을 100으로 정규화해서 비교

In [7]:
def normalize_to_100(df: pd.DataFrame) -> pd.Series:
    """첫 날을 100으로 정규화"""
    s = df["Close"].copy()
    return s / s.iloc[0] * 100


# KOSPI200 vs 단기채권
start_date = max(df_153130.index.min(), df_069500.index.min())
common_index = df_069500.index.intersection(df_153130.index)
df1 = df_069500.loc[common_index]
df2 = df_153130.loc[common_index]

ret1 = (df1["Close"].iloc[-1] / df1["Close"].iloc[0] - 1) * 100
ret2 = (df2["Close"].iloc[-1] / df2["Close"].iloc[0] - 1) * 100

df_plot = pd.DataFrame({
    "Date": common_index,
    "KODEX 200": normalize_to_100(df1),
    "KODEX 단기채": normalize_to_100(df2),
})

fig = px.line(
    df_plot,
    x="Date",
    y=["KODEX 200", "KODEX 단기채"],
    title=(
        "KODEX 200 vs KODEX 단기채 - 정규화 수익률 비교"
        + f"<br><sup>KODEX 200: {ret1:+.2f}%, 단기채: {ret2:+.2f}%</sup>"
    ),
    labels={"value": "정규화 가격(시작=100)", "variable": "ETF"},
)
fig.update_layout(width=900, height=500)
fig.show()

## 5. bt 백테스팅 (Buy & Hold)

### 5-1. 공통 헬퍼 함수

In [8]:
def build_price_dataframe(etf_codes, nav_map):
    """
    etf_codes: ["069500", "153130", ...]
    nav_map: {"069500": df_069500, ...}
    → 각 ETF의 Close를 하나의 DataFrame으로 합치기
    """
    data = pd.DataFrame()
    for code in etf_codes:
        df = nav_map[code][["Close"]].rename(columns={"Close": code})
        data = pd.concat([data, df], axis=1)

    data = data.dropna(how="any")  # 공통 구간만
    return data


def run_buy_hold_backtest(price_df: pd.DataFrame, rf: float = 0.025):
    """
    price_df: index=date, columns = ETF 코드
    - 각 종목을 '사서 보유' 전략으로 개별 백테스트
    """
    strategy = bt.Strategy("Buy & Hold", [
        bt.algos.RunOnce(),
        bt.algos.SelectAll(),
        bt.algos.WeighEqually(),
        bt.algos.Rebalance()
    ])

    backtests = []
    for col in price_df.columns:
        bt_obj = bt.Backtest(strategy, price_df[[col]], name=col)
        backtests.append(bt_obj)

    res = bt.run(*backtests)
    res.set_riskfree_rate(rf)
    return res

### 5-2. 실제 실행 & 결과 해석용 텍스트/마크다운 변환

In [9]:
nav_map = {
    "069500": df_069500,
    "153130": df_153130,
    "329750": df_329750,
}

price_df = build_price_dataframe(["069500", "153130", "329750"], nav_map)
res = run_buy_hold_backtest(price_df, rf=0.025)

# 요약 표 출력 (콘솔)
res.display()

100%|██████████| 3/3 [00:00<00:00, 15.30it/s]


Stat                 069500      153130      329750
-------------------  ----------  ----------  ----------
Start                2019-07-23  2019-07-23  2019-07-23
End                  2025-11-26  2025-11-26  2025-11-26
Risk-free rate       2.50%       2.50%       2.50%

Total Return         131.45%     11.91%      43.18%
Daily Sharpe         0.98        0.86        1.01
Daily Sortino        1.95        13.53       2.16
CAGR                 14.14%      1.79%       5.82%
Max Drawdown         -33.97%     -0.13%      -10.05%
Calmar Ratio         0.42        14.23       0.58

MTD                  -3.61%      0.10%       3.10%
3m                   30.10%      0.40%       5.94%
6m                   68.94%      2.31%       8.90%
YTD                  68.94%      2.31%       8.90%
1Y                   68.94%      2.31%       8.90%
3Y (ann.)            25.39%      2.83%       9.15%
5Y (ann.)            7.90%       2.24%       9.20%
10Y (ann.)           -           -           -
Since Incep. (ann

### 5-3. LLM 프롬프트에 넣기 좋은 문자열로 변환

In [10]:
def backtest_result_to_markdown(res: bt.backtest.Result, code: str) -> str:
    """
    bt 결과 객체에서 특정 코드에 대한 display() 결과를 텍스트로 받아와
    그대로 마크다운 code block으로 감싸기
    """
    # bt의 display는 print 기반이라, StringIO로 우회
    from io import StringIO
    import sys

    old_stdout = sys.stdout
    sys.stdout = mystdout = StringIO()
    try:
        res.display()
    finally:
        sys.stdout = old_stdout

    full_txt = mystdout.getvalue()
    # 여러 전략이 있을 수 있으므로, 코드 이름 있는 줄부터 잘라도 되고
    # 여기서는 그냥 전체를 마크다운으로 감싼다.
    md = "```markdown\n" + full_txt.strip() + "\n```"
    return md


backtest_markdown_all = backtest_result_to_markdown(res, "069500")
print(backtest_markdown_all[:500], "...")

```markdown
Stat                 069500      153130      329750
-------------------  ----------  ----------  ----------
Start                2019-07-23  2019-07-23  2019-07-23
End                  2025-11-26  2025-11-26  2025-11-26
Risk-free rate       2.50%       2.50%       2.50%

Total Return         131.45%     11.91%      43.18%
Daily Sharpe         0.98        0.86        1.01
Daily Sortino        1.95        13.53       2.16
CAGR                 14.14%      1.79%       5.82%
Max Drawdown  ...


## 6. 분석 결과 저장: JSON / CSV / prompt_vars

### 6-1. etf_metrics JSON 스냅샷

In [11]:
def build_etf_metrics(etf_code: str, nav_df: pd.DataFrame, snapshot: dict) -> dict:
    """
    - snapshot: fetch_etf_price_snapshot 결과
    - nav_df: load_or_download_nav 결과 (Close 시계열)
    """
    daily_returns = nav_df["Close"].pct_change().dropna()
    if daily_returns.empty:
        vol_annual = None
        mdd = None
        sharpe = None
        sortino = None
    else:
        vol_annual = daily_returns.std() * np.sqrt(252) * 100
        mdd = qs.stats.max_drawdown(daily_returns) * 100
        sharpe = qs.stats.sharpe(daily_returns, rf=0.025)
        sortino = qs.stats.sortino(daily_returns, rf=0.025)

    metrics = {
        "metadata": {
            "as_of": datetime.now().isoformat(),
            "etf_code": etf_code,
            "etf_name": snapshot["etf_name"],
            "data_source": "KIS API + Calculations"
        },
        "basic_info": {
            "listing_date": snapshot["listing_date"],
            "days_since_listing": snapshot["days_since_listing"],
            "years_since_listing": snapshot["years_since_listing"],
        },
        "price": {
            "current_price": snapshot["current_price"],
            "nav": snapshot["nav"],
            "nav_change": snapshot["nav_change"],
            "nav_change_pct": snapshot["nav_change_pct"],
        },
        "premium_discount": {
            "premium_pct": snapshot["premium_pct"],
        },
        "liquidity": {
            "today_volume": snapshot["volume"],
            "today_turnover_billion": snapshot["turnover_billion"],
        },
        "aum": {
            "aum_total": snapshot["aum_total"],
            "aum_billion": snapshot["aum_billion"],
            "aum_trillion": snapshot["aum_trillion"],
        },
        "risk": {
            "tracking_error_api": snapshot["tracking_error_api"],
            "daily_vol_annual_pct": vol_annual,
            "max_drawdown_pct": mdd,
            "sharpe_ratio": sharpe,
            "sortino_ratio": sortino,
        }
    }
    return metrics


for code in DEFAULT_ETF_CODES:
    nav_df = nav_map[code]
    snap = fetch_etf_price_snapshot(code)
    metrics = build_etf_metrics(code, nav_df, snap)

    out_path = BASE_DATA_DIR / code / f"etf_metrics_{code}_latest.json"
    with open(out_path, "w", encoding="utf-8") as f:
        json.dump(metrics, f, ensure_ascii=False, indent=2)
    print(f"✅ {code} etf_metrics 저장:", out_path)

✅ 기존 토큰 유효함
✅ ETF 이름(MST): 069500 → KODEX 200
✅ 069500 etf_metrics 저장: /Users/ulift/workspace/py-etf-mango/data/069500/etf_metrics_069500_latest.json
✅ 기존 토큰 유효함
✅ ETF 이름(MST): 153130 → KODEX 단기채권
✅ 153130 etf_metrics 저장: /Users/ulift/workspace/py-etf-mango/data/153130/etf_metrics_153130_latest.json
✅ 기존 토큰 유효함
✅ ETF 이름(MST): 329750 → TIGER 미국달러단기채권액티브
✅ 329750 etf_metrics 저장: /Users/ulift/workspace/py-etf-mango/data/329750/etf_metrics_329750_latest.json


### 6-2. 시계열 CSV 업데이트 (시그널·백테스트용)

In [12]:
def append_timeseries_row(etf_code: str, snapshot: dict):
    row = {
        "date": datetime.now().strftime("%Y-%m-%d"),
        "time": datetime.now().strftime("%H:%M:%S"),
        "etf_code": etf_code,
        "price": snapshot["current_price"],
        "nav": snapshot["nav"],
        "premium_pct": snapshot["premium_pct"],
        "volume": snapshot["volume"],
        "turnover_billion": snapshot["turnover_billion"],
        "aum_trillion": snapshot["aum_trillion"],
        "te_pct": snapshot["tracking_error_api"],
    }
    csv_path = BASE_DATA_DIR / etf_code / f"etf_timeseries_{etf_code}.csv"

    if csv_path.exists():
        df_old = pd.read_csv(csv_path)
        df_new = pd.concat([df_old, pd.DataFrame([row])], ignore_index=True)
    else:
        df_new = pd.DataFrame([row])

    df_new.to_csv(csv_path, index=False, encoding="utf-8-sig")
    print(f"✅ 시계열 CSV 업데이트: {csv_path}")


for code in DEFAULT_ETF_CODES:
    snap = fetch_etf_price_snapshot(code)
    append_timeseries_row(code, snap)

✅ 기존 토큰 유효함
✅ ETF 이름(MST): 069500 → KODEX 200
✅ 시계열 CSV 업데이트: /Users/ulift/workspace/py-etf-mango/data/069500/etf_timeseries_069500.csv
✅ 기존 토큰 유효함
✅ ETF 이름(MST): 153130 → KODEX 단기채권
✅ 시계열 CSV 업데이트: /Users/ulift/workspace/py-etf-mango/data/153130/etf_timeseries_153130.csv
✅ 기존 토큰 유효함
✅ ETF 이름(MST): 329750 → TIGER 미국달러단기채권액티브
✅ 시계열 CSV 업데이트: /Users/ulift/workspace/py-etf-mango/data/329750/etf_timeseries_329750.csv


### 6-3. LLM 보고서 생성을 위한 prompt_vars

In [13]:
def build_prompt_vars(etf_code: str, bt_result_markdown: str):
    snap = fetch_etf_price_snapshot(etf_code)
    metrics_path = BASE_DATA_DIR / etf_code / f"etf_metrics_{etf_code}_latest.json"
    with open(metrics_path, "r", encoding="utf-8") as f:
        etf_info = json.load(f)

    prompt_vars = {
        "today": datetime.now().strftime("%Y-%m-%d"),
        "ticker": etf_code,
        "name": snap["etf_name"],
        "etf_info": etf_info,             # JSON 그대로 넘겨도 되고, 필요한 필드만 추려도 됨
        "backtest_result": bt_result_markdown,
    }
    return prompt_vars


# 백테스트 결과를 한 번만 문자열로 만들어서 각 코드에 재사용해도 되고,
# 종목별 개별 backtest를 따로 돌려서 넣어도 됩니다.
bt_md = backtest_result_to_markdown(res, "069500")  # 여기선 전체 결과를 그냥 사용

prompt_vars_stock = build_prompt_vars("069500", bt_md)
prompt_vars_bond  = build_prompt_vars("153130", bt_md)
prompt_vars_us    = build_prompt_vars("329750", bt_md)

for name, pv in [
    ("stock_069500", prompt_vars_stock),
    ("bond_153130", prompt_vars_bond),
    ("us_329750", prompt_vars_us),
]:
    out_path = BASE_DATA_DIR / f"prompt_vars_{name}.json"
    with open(out_path, "w", encoding="utf-8") as f:
        json.dump(pv, f, ensure_ascii=False, indent=2)
    print(f"✅ prompt_vars 저장: {out_path}")

✅ 기존 토큰 유효함
✅ ETF 이름(MST): 069500 → KODEX 200
✅ 기존 토큰 유효함
✅ ETF 이름(MST): 153130 → KODEX 단기채권
✅ 기존 토큰 유효함
✅ ETF 이름(MST): 329750 → TIGER 미국달러단기채권액티브
✅ prompt_vars 저장: /Users/ulift/workspace/py-etf-mango/data/prompt_vars_stock_069500.json
✅ prompt_vars 저장: /Users/ulift/workspace/py-etf-mango/data/prompt_vars_bond_153130.json
✅ prompt_vars 저장: /Users/ulift/workspace/py-etf-mango/data/prompt_vars_us_329750.json


이제 11강에서 설계한 LLM 프롬프트에 바로 이렇게 쓸 수 있습니다:

In [14]:
with open("data/prompt_vars_stock_069500.json", "r", encoding="utf-8") as f:
    prompt_vars_stock = json.load(f)

SYSTEM_MESSAGE = """
당신은 전문 펀드 매니저 겸 리서치 애널리스트입니다.
ETF 투자에 대해 깊이 있는 분석과 실질적인 조언을 제공합니다.
답변은 항상 한국어로 작성하고, Markdown 형식을 활용해 체계적으로 정리하세요.
"""

DEVELOPER_MESSAGE = """
당신은 전문 펀드 매니저로서, ETF 투자에 대해 깊이 있는 분석과 실질적인 조언을 제공합니다.
프롬프트에 주어진 정보와 최신 웹 검색(한국거래소 상장 ETF 및 관련 이슈)에 기반하여,
입문 수준의 투자자를 대상으로 논리적이고 신뢰할 수 있는 답변을 작성하세요.

# 출력 형식
- 답변은 반드시 한국어 작성
- Markdown 구조(제목/표/목록) 필수
- 전문 용어는 한국어로 풀어주기
- 데이터·뉴스는 최신 검색 기준으로 출처/날짜 명시

# 주의사항
- 허위 정보 금지
- 오래된 기사/지표는 사용하지 않음
- 불확실하면 "확인 필요"로 명시
"""

USER_PROMPT_STOCK = f"""
한국거래소에 상장된 주식형 ETF [{prompt_vars_stock['ticker']}] {prompt_vars_stock['name']} 에 대해,
다음 데이터 기반으로 분석 보고서를 작성해 주세요.

[기본정보 JSON]
{json.dumps(prompt_vars_stock['etf_info'], ensure_ascii=False, indent=2)}

[백테스트 결과]
{prompt_vars_stock['backtest_result']}

요구사항:
- A4 1페이지 내외
- 다음 섹션 포함:
  1) 전체 요약
  2) 종목 소개
  3) 수익성과 위험 지표 해설
  4) 종목 장단점
"""


In [15]:
def call_llm_report(user_prompt: str, model: str = "gpt-5.1"):
    response = client.responses.create(
        model=model,
        input=[
            {"role": "system", "content": SYSTEM_MESSAGE},
            {"role": "developer", "content": DEVELOPER_MESSAGE},
            {"role": "user", "content": user_prompt},
        ],
        tools=[{"type": "web_search"}],
    )

    return response.output_text


# 실제 생성
report_stock_069500 = call_llm_report(USER_PROMPT_STOCK)

print(report_stock_069500[:1500])

# 저장
REPORT_DIR = Path("reports")
REPORT_DIR.mkdir(parents=True, exist_ok=True)

out_path = REPORT_DIR / f"report_{prompt_vars_stock['ticker']}_{prompt_vars_stock['today']}.md"
out_path.write_text(report_stock_069500, encoding="utf-8")

print("보고서 저장 완료:", out_path)

## 1) 전체 요약

- **KODEX 200(069500)**은 코스피200 지수를 추종하는 대표적인 국내 주식형 ETF로, **상장 23년 차(2002.10 상장)**에 접어든 대형 패시브 상품입니다.  
- 제시된 기준일(2025-11-26) 기준:
  - **현재가 56,080원**, 순자산가치(NAV) 56,133.76원으로 **프리미엄 -0.10% 수준(미세 할인)**  
  - **순자산 10.8조원**, **일 거래대금 약 5.4조원**, **거래량 965만주**로 국내 ETF 중에서도 **최상위 수준 유동성**  
- 2019-07-23 ~ 2025-11-26 백테스트에서:
  - **누적수익률 131.45%, 연환산 수익률(CAGR) 14.14%**  
  - 연환산 변동성 약 35%, **최대 낙폭 -34%**, 샤프지수 0.98로 **위험은 크지만, 그만큼 보상(수익)도 컸던 전형적인 주식형**  
- 결론적으로, **국내 대형주 시장(코스피200)에 장기 분산투자**하고 싶은 **입문 투자자에게 핵심 코어 ETF**로 적합하되,  
  **주식시장의 큰 변동성과 최대 30% 이상 하락을 감내할 수 있는지**가 투자 판단의 핵심입니다.

---

## 2) 종목 소개

### 2-1. 기본 개요

- **종목명**: KODEX 200  
- **티커**: 069500  
- **상장일**: 2002-10-14 (상장 후 약 23.1년 경과, 8,444거래일)  
- **추종 지수**: 코스피200 지수  
  - 코스피 시장의 시가총액 상위 200개 종목으로 구성된 **대표 벤치마크 지수**  
  - 삼성전자, SK하이닉스, 현대차, LG화학 등 **대형 우량주** 중심  
- **운용사**: 삼성자산운용(패시브 운용 대표 업체, KODEX 브랜드)

### 2-2. 규모 및 유동성

- **총 운용자산(AUM)**: 약 **10.84조원**  
  - `aum_total`: 10,836,622,368,000원  
- **1일 거래대금